In [9]:
import numpy as np

In [10]:
import sys

In [11]:
import pandas as pd

In [12]:
from collections import defaultdict

In [13]:
from Bio import SeqIO

In [14]:
import torch

In [15]:
import time

In [16]:
from torch.utils.data import WeightedRandomSampler

In [17]:
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [3]:
a = list(range(10))

In [2]:
np.array([3.0]).astype(int)

array([3])

In [4]:
sys.path

['C:\\Users\\user\\anaconda3\\envs\\DeepMicrobes\\python36.zip',
 'C:\\Users\\user\\anaconda3\\envs\\DeepMicrobes\\DLLs',
 'C:\\Users\\user\\anaconda3\\envs\\DeepMicrobes\\lib',
 'C:\\Users\\user\\anaconda3\\envs\\DeepMicrobes',
 '',
 'C:\\Users\\user\\anaconda3\\envs\\DeepMicrobes\\lib\\site-packages',
 'C:\\Users\\user\\anaconda3\\envs\\DeepMicrobes\\lib\\site-packages\\win32',
 'C:\\Users\\user\\anaconda3\\envs\\DeepMicrobes\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\user\\anaconda3\\envs\\DeepMicrobes\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\user\\anaconda3\\envs\\DeepMicrobes\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\user\\.ipython']

In [18]:
def extrac_subtype_seq(fileName,outfile, filetype='fasta'):
    subtypes = []
    seqs = []
    fh = open(outfile, 'w')
    
    with open(fileName) as handle:
        for rec in SeqIO.parse(handle,filetype):
            seq = str(rec.seq)
            seq = seq.replace('-','').upper()
            
            label_id = rec.id
            ls = label_id.split('.')
            label_id = '>' + '_'.join((ls[-1], ls[0]))

            fh.write(label_id)
            fh.write('\n')
            fh.write(seq)
            fh.write('\n')
            
    fh.close()
                    

def sort_labels(labels):
    labels = np.array(labels)

    labels_dict = defaultdict(int)

    for i, v in enumerate(labels):
        labels_dict[v] += 1

    labels_num = [(k,v) for k, v in labels_dict.items()]

    labels_num_sort = sorted(labels_num, key=lambda x:x[1], reverse=True)
    
    return labels_num_sort

def combine_labels(labels_sorted):
    labels_com = defaultdict(int)
    combined_labels = []
    for( k,v) in labels_sorted:
        ls = k.split('_')
        if len(ls) ==3:
            combined_labels.append('_'.join((ls[0],ls[-1],ls[1])))
            k = "_".join((ls[0], ls[-1]))
            
        labels_com[k] += v
        
    labels_num_sort = sorted([(k,v) for k,v in labels_com.items()], key=lambda x:x[1], reverse=True)
        
    return labels_num_sort, combined_labels
        

def get_lgn(labels_sorted, n):
    return [(k,v) for k,v in labels_sorted if v >= n]

In [19]:
class MyDataset(Dataset):
    def __init__(self, features, labels, labelsDicFile):
        self.X = torch.LongTensor(features )
        self.labels  = labels
        self.labelsDicFile = self.read_dic(labelsDicFile)
        self.labels_num = torch.Tensor([self.labelsDicFile[label] for label in self.labels] )
        
        
    def read_dic(self,dic_file):
        labels_dic = {}
        with(open(dic_file, 'r')) as fh:
            for line in fh:
                label, label_id = line.strip().split()
                labels_dic[label] = int(label_id)
        return labels_dic
    
    
    def __getitem__(self, index):
        return self.X[index], self.labels_num[index]

    def get_output_dim(self):
        return len(self.labelsDicFile.keys())
    
    def __len__(self):
        return len(self.labels)

class Subset(Dataset):
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices
        
    def __getitem__(self,idx):
        return self.dataset[self.indices[idx]]
    
    def __len__(self):
        return len(self.indices)

In [20]:
def get_sample_weights(labelsFile):
    labels = np.load(labelsFile)

    labels_dict = defaultdict(list)

    for i, v in enumerate(labels):
        labels_dict[v].append(i)

    labels_num = [(k,len(v)) for k, v in labels_dict.items()]

    labels_num_sort = sorted(labels_num, key=lambda x:x[1], reverse=True)

    total_weights = len(labels)

    labels_weight = [(k, total_weights / num) for k, num in labels_num_sort ]

    labels_weight_dict = {}
    for k, v in labels_weight:
        labels_weight_dict[k] = v

    sample_weights= [ labels_weight_dict[k] for k in labels]
    
    return np.array(sample_weights)

In [21]:
def read_fasta_labels(fastaFile):
    labels = []
    for rec in SeqIO.parse(fastaFile,'fasta'):
        label_id = rec.id
        labels.append(label_id)
        
    return labels
       

In [22]:
fasta_100bp = './hiv_12825/hiv_100bp_600.fasta'

In [23]:
labels_100bp = read_fasta_labels(fasta_100bp)

In [24]:
len(labels_100bp)

1924485

In [25]:
labels_sorted = sort_labels(labels_100bp)

In [26]:
labels_sorted

[('HIV-1_B', 53585),
 ('HIV-1_C', 53217),
 ('HIV-1_01_AE', 52848),
 ('HIV-1_O', 49020),
 ('HIV-1_U', 48346),
 ('HIV-1_D', 47644),
 ('HIV-1_F1', 47574),
 ('HIV-1_A1', 47287),
 ('HIV-1_G', 47234),
 ('HIV-1_01B', 47223),
 ('HIV-1_02A1', 47195),
 ('HIV-1_A6', 47120),
 ('HIV-1_02_AG', 47031),
 ('HIV-1_BC', 46931),
 ('HIV-1_BF1', 46877),
 ('HIV-1_A1C', 46058),
 ('HIV-1_H', 46022),
 ('HIV-1_A1D', 45629),
 ('HIV-1_07_BC', 45497),
 ('HIV-1_08_BC', 45132),
 ('HIV-1_CD', 44977),
 ('HIV-1_AD', 44644),
 ('HIV-1_11_cpx', 44502),
 ('HIV-1_A1CD', 44374),
 ('HIV-1_13_cpx', 44308),
 ('HIV-1_01BC', 43935),
 ('HIV-1_02G', 43885),
 ('HIV-1_06_cpx', 43511),
 ('HIV-1_0107', 43199),
 ('HIV-1_12_BF', 42153),
 ('HIV-1_01C', 41805),
 ('HIV-1_02A', 41598),
 ('HIV-1_BG', 40935),
 ('HIV-1_63_02A', 39625),
 ('HIV-1_22_01A1', 39169),
 ('HIV-1_14_BG', 38439),
 ('HIV-1_ACD', 38299),
 ('HIV-1_71_BF1', 37905),
 ('HIV-1_35_AD', 37781),
 ('HIV-1_42_BF', 37694),
 ('HIV-1_02B', 37296),
 ('HIV-1_85_BC', 32529),
 ('HIV-1_N', 3

In [16]:
originalfile = './hiv-db.fasta'

In [14]:
removeDashFile = './hiv_12825.fasta'

In [95]:
balanced_file = './hiv_12825/hiv_500bp_balanced.fasta'

In [106]:
unbalanced_file  = './hiv_12825/hiv_500.fasta'

In [17]:
balanced_file_less = 'hiv_12825/hiv_500bp_600.fasta'

### HIV1亚型分类结果：原始，合并后

In [18]:
balanced_labels_less = read_fasta_labels(balanced_file_less)

In [19]:
len(balanced_labels_less)

410925

In [20]:
sorted_balanced_labels_less = sort_labels(balanced_labels_less)

In [21]:
len(sorted_balanced_labels_less)

45

In [22]:
sorted_balanced_labels_less

[('HIV-1_C', 10503),
 ('HIV-1_B', 10488),
 ('HIV-1_01_AE', 10371),
 ('HIV-1_O', 9764),
 ('HIV-1_-', 9672),
 ('HIV-1_06_cpx', 9614),
 ('HIV-1_63_02A', 9477),
 ('HIV-1_02A1', 9408),
 ('HIV-1_12_BF', 9303),
 ('HIV-1_G', 9269),
 ('HIV-1_U', 9259),
 ('HIV-1_11_cpx', 9249),
 ('HIV-1_F1', 9248),
 ('HIV-1_A6', 9186),
 ('HIV-1_14_BG', 9181),
 ('HIV-1_BF1', 9176),
 ('HIV-1_08_BC', 9166),
 ('HIV-1_07_BC', 9163),
 ('HIV-1_D', 9138),
 ('HIV-1_01BC', 9133),
 ('HIV-1_BC', 9105),
 ('HIV-1_CD', 9066),
 ('HIV-1_02_AG', 9064),
 ('HIV-1_01B', 9058),
 ('HIV-1_A1D', 9048),
 ('HIV-1_71_BF1', 9048),
 ('HIV-1_BG', 9018),
 ('HIV-1_42_BF', 9005),
 ('HIV-1_0107', 8987),
 ('HIV-1_02G', 8981),
 ('HIV-1_A1C', 8975),
 ('HIV-1_A1CD', 8927),
 ('HIV-1_02B', 8895),
 ('HIV-1_01C', 8876),
 ('HIV-1_AD', 8841),
 ('HIV-1_02A', 8827),
 ('HIV-1_H', 8809),
 ('HIV-1_A1', 8803),
 ('HIV-1_22_01A1', 8801),
 ('HIV-1_ACD', 8710),
 ('HIV-1_35_AD', 8585),
 ('HIV-1_85_BC', 8475),
 ('HIV-1_13_cpx', 8467),
 ('HIV-1_N', 8408),
 ('HIV-1_F2',

In [107]:
unbalanced_labels = read_fasta_labels(unbalanced_file)

In [108]:
sorted_unbalanced_labels = sort_labels(unbalanced_labels)

In [109]:
sorted_unbalanced_labels

[('HIV-1_B', 99414),
 ('HIV-1_C', 36323),
 ('HIV-1_01_AE', 24645),
 ('HIV-1_-', 9673),
 ('HIV-1_A1', 8553),
 ('HIV-1_01B', 3527),
 ('HIV-1_02_AG', 2764),
 ('HIV-1_BF1', 2478),
 ('HIV-1_A6', 2023),
 ('HIV-1_A1C', 1904),
 ('HIV-1_G', 1671),
 ('HIV-1_A1D', 1605),
 ('HIV-1_AD', 1597),
 ('HIV-1_BC', 1565),
 ('HIV-1_D', 1489),
 ('HIV-1_F1', 1295),
 ('HIV-1_O', 1059),
 ('HIV-1_CD', 1058),
 ('HIV-1_A1CD', 1056),
 ('HIV-1_01BC', 862),
 ('HIV-1_0107', 841),
 ('HIV-1_07_BC', 708),
 ('HIV-1_08_BC', 621),
 ('HIV-1_02A1', 468),
 ('HIV-1_11_cpx', 436),
 ('HIV-1_U', 381),
 ('HIV-1_35_AD', 374),
 ('HIV-1_ACD', 374),
 ('HIV-1_01C', 326),
 ('HIV-1_02A', 326),
 ('HIV-1_22_01A1', 306),
 ('HIV-1_42_BF', 299),
 ('HIV-1_02G', 258),
 ('HIV-1_06_cpx', 256),
 ('HIV-1_12_BF', 248),
 ('HIV-1_BG', 239),
 ('HIV-1_63_02A', 234),
 ('HIV-1_14_BG', 226),
 ('HIV-1_71_BF1', 225),
 ('HIV-1_02B', 218),
 ('HIV-1_N', 191),
 ('HIV-1_F2', 190),
 ('HIV-1_85_BC', 189),
 ('HIV-1_H', 181),
 ('HIV-1_13_cpx', 173),
 ('HIV-1_0206', 17

In [113]:
len(unbalanced_labels)

223308

In [132]:
np.random.randn(2,3)

array([[-1.20078684, -1.34142953,  0.41946543],
       [ 0.42033369, -2.49225404, -0.17882419]])

In [114]:
50000*45

2250000

In [96]:
balanced_labels = read_fasta_labels(balanced_file)

In [99]:
len(balanced_labels)

3723927

In [100]:
sorted_balanced_labels = sort_labels(balanced_labels)

In [102]:
len(sorted_balanced_labels)

45

In [101]:
sorted_balanced_labels

[('HIV-1_O', 103252),
 ('HIV-1_B', 99399),
 ('HIV-1_-', 96745),
 ('HIV-1_C', 96297),
 ('HIV-1_01_AE', 96247),
 ('HIV-1_BC', 95136),
 ('HIV-1_01B', 95017),
 ('HIV-1_A1', 94053),
 ('HIV-1_85_BC', 93172),
 ('HIV-1_BF1', 92870),
 ('HIV-1_U', 92478),
 ('HIV-1_N', 92431),
 ('HIV-1_A1C', 92225),
 ('HIV-1_F2', 92104),
 ('HIV-1_ACD', 91320),
 ('HIV-1_D', 90948),
 ('HIV-1_F1', 90824),
 ('HIV-1_G', 90197),
 ('HIV-1_35_AD', 90036),
 ('HIV-1_A6', 89943),
 ('HIV-1_02_AG', 89720),
 ('HIV-1_H', 88037),
 ('HIV-1_A1D', 87028),
 ('HIV-1_07_BC', 86874),
 ('HIV-1_CD', 85859),
 ('HIV-1_AD', 85039),
 ('HIV-1_13_cpx', 84613),
 ('HIV-1_A1CD', 84599),
 ('HIV-1_01BC', 83846),
 ('HIV-1_0107', 82453),
 ('HIV-1_01C', 79799),
 ('HIV-1_02A', 79384),
 ('HIV-1_08_BC', 75470),
 ('HIV-1_02A1', 75181),
 ('HIV-1_22_01A1', 74728),
 ('HIV-1_42_BF', 71978),
 ('HIV-1_11_cpx', 70826),
 ('HIV-1_02G', 62825),
 ('HIV-1_06_cpx', 62464),
 ('HIV-1_12_BF', 60426),
 ('HIV-1_BG', 58597),
 ('HIV-1_63_02A', 56838),
 ('HIV-1_14_BG', 55058)

In [111]:
53338*0.2

10667.6

In [112]:
 99399*0.2

19879.800000000003

In [34]:
original_labels = read_fasta_labels(originalfile)

In [36]:
len(original_labels)

12825

In [15]:
removeDash_labels = read_fasta_labels(removeDashFile)

In [16]:
len(removeDash_labels)

12825

In [17]:
sorted_removeDash_labels = sort_labels(removeDash_labels)

In [18]:
gt10_removeDash_labels = get_lgn(sorted_removeDash_labels,10)

In [19]:
len(gt10_removeDash_labels)

45

In [21]:
gt10_removeDash_labels

[('HIV-1_B', 5703),
 ('HIV-1_C', 2077),
 ('HIV-1_01_AE', 1425),
 ('HIV-1_-', 544),
 ('HIV-1_A1', 497),
 ('HIV-1_01B', 205),
 ('HIV-1_02_AG', 160),
 ('HIV-1_BF1', 143),
 ('HIV-1_A6', 117),
 ('HIV-1_A1C', 111),
 ('HIV-1_G', 95),
 ('HIV-1_A1D', 94),
 ('HIV-1_AD', 94),
 ('HIV-1_BC', 91),
 ('HIV-1_D', 86),
 ('HIV-1_F1', 75),
 ('HIV-1_A1CD', 62),
 ('HIV-1_CD', 61),
 ('HIV-1_O', 57),
 ('HIV-1_01BC', 50),
 ('HIV-1_0107', 49),
 ('HIV-1_07_BC', 41),
 ('HIV-1_08_BC', 35),
 ('HIV-1_02A1', 26),
 ('HIV-1_11_cpx', 25),
 ('HIV-1_35_AD', 22),
 ('HIV-1_ACD', 22),
 ('HIV-1_U', 21),
 ('HIV-1_01C', 19),
 ('HIV-1_02A', 19),
 ('HIV-1_22_01A1', 18),
 ('HIV-1_42_BF', 17),
 ('HIV-1_02G', 15),
 ('HIV-1_06_cpx', 14),
 ('HIV-1_12_BF', 14),
 ('HIV-1_BG', 14),
 ('HIV-1_14_BG', 13),
 ('HIV-1_71_BF1', 13),
 ('HIV-1_02B', 13),
 ('HIV-1_63_02A', 13),
 ('HIV-1_F2', 11),
 ('HIV-1_N', 11),
 ('HIV-1_85_BC', 11),
 ('HIV-1_H', 10),
 ('HIV-1_13_cpx', 10)]

In [25]:
total = 0
labels_num = [v  for (k,v) in gt10_removeDash_labels]

In [115]:
total = np.sum(labels_num)
mark = 544

In [116]:
dict_desire_labels = [(k,mark/v) for k,v in gt10_removeDash_labels]

In [117]:
dict_desire_labels

[('HIV-1_B', 0.09538839207434684),
 ('HIV-1_C', 0.261916225324988),
 ('HIV-1_01_AE', 0.3817543859649123),
 ('HIV-1_-', 1.0),
 ('HIV-1_A1', 1.0945674044265594),
 ('HIV-1_01B', 2.653658536585366),
 ('HIV-1_02_AG', 3.4),
 ('HIV-1_BF1', 3.804195804195804),
 ('HIV-1_A6', 4.64957264957265),
 ('HIV-1_A1C', 4.900900900900901),
 ('HIV-1_G', 5.726315789473684),
 ('HIV-1_A1D', 5.787234042553192),
 ('HIV-1_AD', 5.787234042553192),
 ('HIV-1_BC', 5.978021978021978),
 ('HIV-1_D', 6.325581395348837),
 ('HIV-1_F1', 7.253333333333333),
 ('HIV-1_A1CD', 8.774193548387096),
 ('HIV-1_CD', 8.918032786885245),
 ('HIV-1_O', 9.543859649122806),
 ('HIV-1_01BC', 10.88),
 ('HIV-1_0107', 11.10204081632653),
 ('HIV-1_07_BC', 13.268292682926829),
 ('HIV-1_08_BC', 15.542857142857143),
 ('HIV-1_02A1', 20.923076923076923),
 ('HIV-1_11_cpx', 21.76),
 ('HIV-1_35_AD', 24.727272727272727),
 ('HIV-1_ACD', 24.727272727272727),
 ('HIV-1_U', 25.904761904761905),
 ('HIV-1_01C', 28.63157894736842),
 ('HIV-1_02A', 28.6315789473684

In [118]:
ofile = './hiv_12825/dict_desire_labels_less.txt'

In [119]:
fh = open(ofile,'w')

In [120]:
for k, v in dict_desire_labels:
    fh.write(k)
    fh.write(' ')
    fh.write(str(v))
    fh.write('\n')

In [121]:
fh.close()

In [39]:
np.ceil(500/570)

1.0

In [20]:
sorted_removeDash_labels

[('HIV-1_B', 5703),
 ('HIV-1_C', 2077),
 ('HIV-1_01_AE', 1425),
 ('HIV-1_-', 544),
 ('HIV-1_A1', 497),
 ('HIV-1_01B', 205),
 ('HIV-1_02_AG', 160),
 ('HIV-1_BF1', 143),
 ('HIV-1_A6', 117),
 ('HIV-1_A1C', 111),
 ('HIV-1_G', 95),
 ('HIV-1_A1D', 94),
 ('HIV-1_AD', 94),
 ('HIV-1_BC', 91),
 ('HIV-1_D', 86),
 ('HIV-1_F1', 75),
 ('HIV-1_A1CD', 62),
 ('HIV-1_CD', 61),
 ('HIV-1_O', 57),
 ('HIV-1_01BC', 50),
 ('HIV-1_0107', 49),
 ('HIV-1_07_BC', 41),
 ('HIV-1_08_BC', 35),
 ('HIV-1_02A1', 26),
 ('HIV-1_11_cpx', 25),
 ('HIV-1_35_AD', 22),
 ('HIV-1_ACD', 22),
 ('HIV-1_U', 21),
 ('HIV-1_01C', 19),
 ('HIV-1_02A', 19),
 ('HIV-1_22_01A1', 18),
 ('HIV-1_42_BF', 17),
 ('HIV-1_02G', 15),
 ('HIV-1_06_cpx', 14),
 ('HIV-1_12_BF', 14),
 ('HIV-1_BG', 14),
 ('HIV-1_14_BG', 13),
 ('HIV-1_71_BF1', 13),
 ('HIV-1_02B', 13),
 ('HIV-1_63_02A', 13),
 ('HIV-1_F2', 11),
 ('HIV-1_N', 11),
 ('HIV-1_85_BC', 11),
 ('HIV-1_H', 10),
 ('HIV-1_13_cpx', 10),
 ('HIV-1_0206', 9),
 ('HIV-1_59_01B', 9),
 ('HIV-1_64_BC', 9),
 ('HIV-1_

In [43]:
len(sorted_removeDash_labels)

242

In [52]:
combine_removeDash_labels, combined_labels = combine_labels(sorted_removeDash_labels)

In [53]:
len(combine_removeDash_labels)

156

In [68]:
lg10_combine_removeDash_labels = get_lgn(combine_removeDash_labels,10)

In [69]:
len(lg10_combine_removeDash_labels)

35

In [70]:
lg10_combine_removeDash_labels

[('HIV-1_B', 5703),
 ('HIV-1_C', 2077),
 ('HIV-1_AE', 1425),
 ('HIV-1_-', 544),
 ('HIV-1_A1', 497),
 ('HIV-1_01B', 288),
 ('HIV-1_BC', 212),
 ('HIV-1_BF1', 179),
 ('HIV-1_AG', 160),
 ('HIV-1_cpx', 127),
 ('HIV-1_A6', 117),
 ('HIV-1_AD', 116),
 ('HIV-1_A1C', 111),
 ('HIV-1_A1D', 102),
 ('HIV-1_G', 95),
 ('HIV-1_D', 86),
 ('HIV-1_BF', 82),
 ('HIV-1_F1', 75),
 ('HIV-1_CD', 67),
 ('HIV-1_A1CD', 62),
 ('HIV-1_O', 57),
 ('HIV-1_0107', 56),
 ('HIV-1_01BC', 50),
 ('HIV-1_BG', 41),
 ('HIV-1_02A', 32),
 ('HIV-1_02A1', 26),
 ('HIV-1_ACD', 22),
 ('HIV-1_01C', 22),
 ('HIV-1_U', 21),
 ('HIV-1_02G', 20),
 ('HIV-1_01A1', 18),
 ('HIV-1_02B', 16),
 ('HIV-1_F2', 11),
 ('HIV-1_N', 11),
 ('HIV-1_H', 10)]

In [71]:
gt10_removeDash_labels

[('HIV-1_B', 5703),
 ('HIV-1_C', 2077),
 ('HIV-1_01_AE', 1425),
 ('HIV-1_-', 544),
 ('HIV-1_A1', 497),
 ('HIV-1_01B', 205),
 ('HIV-1_02_AG', 160),
 ('HIV-1_BF1', 143),
 ('HIV-1_A6', 117),
 ('HIV-1_A1C', 111),
 ('HIV-1_G', 95),
 ('HIV-1_A1D', 94),
 ('HIV-1_AD', 94),
 ('HIV-1_BC', 91),
 ('HIV-1_D', 86),
 ('HIV-1_F1', 75),
 ('HIV-1_A1CD', 62),
 ('HIV-1_CD', 61),
 ('HIV-1_O', 57),
 ('HIV-1_01BC', 50),
 ('HIV-1_0107', 49),
 ('HIV-1_07_BC', 41),
 ('HIV-1_08_BC', 35),
 ('HIV-1_02A1', 26),
 ('HIV-1_11_cpx', 25),
 ('HIV-1_35_AD', 22),
 ('HIV-1_ACD', 22),
 ('HIV-1_U', 21),
 ('HIV-1_01C', 19),
 ('HIV-1_02A', 19),
 ('HIV-1_22_01A1', 18),
 ('HIV-1_42_BF', 17),
 ('HIV-1_02G', 15),
 ('HIV-1_06_cpx', 14),
 ('HIV-1_12_BF', 14),
 ('HIV-1_BG', 14),
 ('HIV-1_14_BG', 13),
 ('HIV-1_71_BF1', 13),
 ('HIV-1_02B', 13),
 ('HIV-1_63_02A', 13),
 ('HIV-1_F2', 11),
 ('HIV-1_N', 11),
 ('HIV-1_85_BC', 11),
 ('HIV-1_H', 10),
 ('HIV-1_13_cpx', 10)]

In [55]:
sorted(combined_labels)

['HIV-1_0107_102',
 'HIV-1_0107_79',
 'HIV-1_0107_80',
 'HIV-1_01A1_22',
 'HIV-1_01B_103',
 'HIV-1_01B_15',
 'HIV-1_01B_33',
 'HIV-1_01B_34',
 'HIV-1_01B_48',
 'HIV-1_01B_51',
 'HIV-1_01B_52',
 'HIV-1_01B_53',
 'HIV-1_01B_54',
 'HIV-1_01B_55',
 'HIV-1_01B_58',
 'HIV-1_01B_59',
 'HIV-1_01B_67',
 'HIV-1_01B_68',
 'HIV-1_01B_69',
 'HIV-1_01B_74',
 'HIV-1_01B_76',
 'HIV-1_01C_100',
 'HIV-1_02A_63',
 'HIV-1_02B_95',
 'HIV-1_02G_43',
 'HIV-1_06A6_32',
 'HIV-1_06B_98',
 'HIV-1_A1D_50',
 'HIV-1_A1D_84',
 'HIV-1_A2D_16',
 'HIV-1_A2D_21',
 'HIV-1_A5U_26',
 'HIV-1_AB_03',
 'HIV-1_AD_35',
 'HIV-1_AE_01',
 'HIV-1_AG_02',
 'HIV-1_BC_07',
 'HIV-1_BC_08',
 'HIV-1_BC_31',
 'HIV-1_BC_57',
 'HIV-1_BC_60',
 'HIV-1_BC_61',
 'HIV-1_BC_62',
 'HIV-1_BC_64',
 'HIV-1_BC_85',
 'HIV-1_BC_88',
 'HIV-1_BF1_38',
 'HIV-1_BF1_66',
 'HIV-1_BF1_70',
 'HIV-1_BF1_71',
 'HIV-1_BF1_72',
 'HIV-1_BF1_90',
 'HIV-1_BF_12',
 'HIV-1_BF_17',
 'HIV-1_BF_28',
 'HIV-1_BF_29',
 'HIV-1_BF_38',
 'HIV-1_BF_39',
 'HIV-1_BF_40',
 'HIV-1_BF

In [46]:
combine_removeDash_labels

[('HIV-1_B', 5703),
 ('HIV-1_C', 2077),
 ('HIV-1_AE', 1425),
 ('HIV-1_-', 544),
 ('HIV-1_A1', 497),
 ('HIV-1_01B', 288),
 ('HIV-1_BC', 212),
 ('HIV-1_BF1', 179),
 ('HIV-1_AG', 160),
 ('HIV-1_cpx', 127),
 ('HIV-1_A6', 117),
 ('HIV-1_AD', 116),
 ('HIV-1_A1C', 111),
 ('HIV-1_A1D', 102),
 ('HIV-1_G', 95),
 ('HIV-1_D', 86),
 ('HIV-1_BF', 82),
 ('HIV-1_F1', 75),
 ('HIV-1_CD', 67),
 ('HIV-1_A1CD', 62),
 ('HIV-1_O', 57),
 ('HIV-1_0107', 56),
 ('HIV-1_01BC', 50),
 ('HIV-1_BG', 41),
 ('HIV-1_02A', 32),
 ('HIV-1_02A1', 26),
 ('HIV-1_ACD', 22),
 ('HIV-1_01C', 22),
 ('HIV-1_U', 21),
 ('HIV-1_02G', 20),
 ('HIV-1_01A1', 18),
 ('HIV-1_02B', 16),
 ('HIV-1_F2', 11),
 ('HIV-1_N', 11),
 ('HIV-1_H', 10),
 ('HIV-1_0206', 9),
 ('HIV-1_A', 8),
 ('HIV-1_02A3', 7),
 ('HIV-1_J', 7),
 ('HIV-1_AB', 7),
 ('HIV-1_A2D', 7),
 ('HIV-1_02D', 6),
 ('HIV-1_0209', 6),
 ('HIV-1_0222', 6),
 ('HIV-1_02A6', 6),
 ('HIV-1_CU', 5),
 ('HIV-1_A1U', 5),
 ('HIV-1_A1G', 5),
 ('HIV-1_P', 5),
 ('HIV-1_A1B', 5),
 ('HIV-1_02F2', 5),
 ('HI

In [49]:
labelsDicFile = './hiv_12825/labels_dict_31.txt'

In [72]:
labels = np.load('./hiv_12825/12mer_500_labels_2020-07-10-14_34_20.npy')

In [66]:
features = np.load('./hiv_12825/12mer_500_features_index.npy')

In [67]:
dataset = MyDataset(features,labels,labelsDicFile)

In [73]:
sorted_labels = sort_labels(labels)

In [76]:
sorted_labels

[('HIV-1_B', 99414),
 ('HIV-1_C', 36323),
 ('HIV-1_AE', 24645),
 ('HIV-1_-', 9673),
 ('HIV-1_A1', 8553),
 ('HIV-1_01B', 4954),
 ('HIV-1_BC', 3677),
 ('HIV-1_BF1', 3100),
 ('HIV-1_AG', 2764),
 ('HIV-1_cpx', 2211),
 ('HIV-1_A6', 2023),
 ('HIV-1_AD', 1971),
 ('HIV-1_A1C', 1904),
 ('HIV-1_A1D', 1741),
 ('HIV-1_G', 1671),
 ('HIV-1_D', 1489),
 ('HIV-1_BF', 1426),
 ('HIV-1_F1', 1295),
 ('HIV-1_CD', 1161),
 ('HIV-1_O', 1059),
 ('HIV-1_A1CD', 1056),
 ('HIV-1_0107', 962),
 ('HIV-1_01BC', 862),
 ('HIV-1_BG', 708),
 ('HIV-1_02A', 560),
 ('HIV-1_02A1', 468),
 ('HIV-1_U', 381),
 ('HIV-1_01C', 377),
 ('HIV-1_ACD', 374),
 ('HIV-1_02G', 346),
 ('HIV-1_01A1', 306),
 ('HIV-1_02B', 269),
 ('HIV-1_N', 191),
 ('HIV-1_F2', 190),
 ('HIV-1_H', 181)]

In [77]:
len(sorted_labels)

35

In [85]:
fh = open('./hiv_12825/labels_dict_35.txt','w')

In [86]:
for i, (k,v) in enumerate(sorted_labels):
    fh.write(k)
    fh.write(' ')
    fh.write(str(i+1))
    fh.write('\n')

In [87]:
fh.close()

In [22]:
total = len(labels)
total

209722

In [39]:
weights = [ (k,total/n)  for k,n in sorted_labels]
weights_dict = {}
for k,w in weights:
    weights_dict[k]=w

In [40]:
labels_weigth

{'HIV-1_B': 2.1095821514072464,
 'HIV-1_C': 5.773807229579055,
 'HIV-1_01_AE': 8.50971799553662,
 'HIV-1_-': 21.68117440297736,
 'HIV-1_A1': 24.520285280018708,
 'HIV-1_01B': 59.46186560816558,
 'HIV-1_02_AG': 75.87626628075253,
 'HIV-1_BF1': 84.63357546408393,
 'HIV-1_A6': 103.66880869995057,
 'HIV-1_A1C': 110.14810924369748,
 'HIV-1_G': 125.50688210652304,
 'HIV-1_A1D': 130.66791277258568,
 'HIV-1_AD': 131.3224796493425,
 'HIV-1_BC': 134.0076677316294,
 'HIV-1_D': 140.84754869039625,
 'HIV-1_F1': 161.94749034749034,
 'HIV-1_O': 198.0377714825307,
 'HIV-1_CD': 198.2249527410208,
 'HIV-1_A1CD': 198.60037878787878,
 'HIV-1_01BC': 243.2969837587007,
 'HIV-1_0107': 249.37217598097504,
 'HIV-1_07_BC': 296.21751412429376,
 'HIV-1_08_BC': 337.71658615136874,
 'HIV-1_02A1': 448.12393162393164,
 'HIV-1_11_cpx': 481.01376146788994,
 'HIV-1_U': 550.4514435695538,
 'HIV-1_35_AD': 560.7540106951872,
 'HIV-1_ACD': 560.7540106951872,
 'HIV-1_01C': 643.319018404908,
 'HIV-1_02A': 643.319018404908,
 '

In [73]:
20*0.8

16.0

In [45]:
sample_weights = get_sample_weights('./hiv_12825/12mer_500_labels.npy')

In [68]:
sampler = WeightedRandomSampler(sample_weights,\
                                num_samples=100000,\
                                replacement=True)

In [69]:
dataloader = DataLoader(dataset,
                        batch_size=4000,
                        sampler=sampler)

In [72]:
for datas, labels in dataloader:
    print(sort_labels(labels))
    print(len(np.unique(labels)))

[(26.0, 143), (3.0, 142), (1.0, 141), (7.0, 141), (12.0, 140), (2.0, 140), (4.0, 139), (10.0, 138), (19.0, 137), (27.0, 135), (6.0, 135), (23.0, 134), (20.0, 134), (5.0, 131), (25.0, 131), (14.0, 131), (16.0, 131), (17.0, 129), (21.0, 127), (15.0, 127), (28.0, 126), (30.0, 124), (18.0, 124), (9.0, 119), (13.0, 118), (31.0, 117), (22.0, 116), (8.0, 115), (29.0, 115), (11.0, 110), (24.0, 110)]
31
[(4.0, 159), (28.0, 153), (29.0, 149), (13.0, 147), (7.0, 145), (15.0, 145), (12.0, 136), (17.0, 136), (14.0, 134), (3.0, 134), (31.0, 133), (30.0, 133), (2.0, 131), (22.0, 131), (11.0, 130), (1.0, 129), (9.0, 128), (18.0, 125), (6.0, 124), (16.0, 123), (8.0, 123), (10.0, 123), (27.0, 119), (24.0, 118), (26.0, 117), (25.0, 117), (21.0, 115), (5.0, 114), (20.0, 113), (23.0, 109), (19.0, 107)]
31
[(7.0, 154), (5.0, 149), (20.0, 143), (2.0, 141), (18.0, 141), (6.0, 139), (23.0, 138), (27.0, 138), (29.0, 136), (31.0, 136), (15.0, 135), (1.0, 135), (17.0, 134), (26.0, 133), (24.0, 133), (30.0, 132), 

[(27.0, 161), (14.0, 156), (10.0, 151), (17.0, 151), (2.0, 140), (31.0, 139), (9.0, 138), (28.0, 135), (5.0, 135), (3.0, 132), (26.0, 131), (22.0, 130), (23.0, 128), (29.0, 128), (30.0, 127), (15.0, 126), (21.0, 125), (20.0, 124), (4.0, 124), (13.0, 123), (18.0, 122), (1.0, 122), (11.0, 121), (19.0, 120), (24.0, 120), (16.0, 118), (8.0, 116), (7.0, 115), (6.0, 115), (25.0, 114), (12.0, 113)]
31


In [38]:
fh = open('./hiv_12825/labels_dict_31.txt', 'w')

In [39]:
for i, k in enumerate(np.unique(labels)):
    fh. write(k)
    fh.write(' ')
    fh.write(str(i+1))
    fh.write('\n')

In [40]:
fh.close()

In [3]:
input_seq = '../../hiv-db.fasta'

In [17]:
output_file = './hiv_12825.fasta'

In [ ]:
file = './hiv'

In [22]:
extrac_subtype_seq(input_seq,output_file)

In [23]:
25650/2

12825.0

In [27]:
i = 0
seq = None
labels= []
with open(output_file) as handle:
    for rec in SeqIO.parse(handle,'fasta'):
        label_id  = rec.id
        labels.append(label_id)
        
               

In [28]:
len(labels)

12825

In [44]:
labels_sorted = sort_labels(labels)

In [57]:
labels_sorted

[('HIV-1_B', 5703),
 ('HIV-1_C', 2077),
 ('HIV-1_01_AE', 1425),
 ('HIV-1_-', 544),
 ('HIV-1_A1', 497),
 ('HIV-1_01B', 205),
 ('HIV-1_02_AG', 160),
 ('HIV-1_BF1', 143),
 ('HIV-1_A6', 117),
 ('HIV-1_A1C', 111),
 ('HIV-1_G', 95),
 ('HIV-1_A1D', 94),
 ('HIV-1_AD', 94),
 ('HIV-1_BC', 91),
 ('HIV-1_D', 86),
 ('HIV-1_F1', 75),
 ('HIV-1_A1CD', 62),
 ('HIV-1_CD', 61),
 ('HIV-1_O', 57),
 ('HIV-1_01BC', 50),
 ('HIV-1_0107', 49),
 ('HIV-1_07_BC', 41),
 ('HIV-1_08_BC', 35),
 ('HIV-1_02A1', 26),
 ('HIV-1_11_cpx', 25),
 ('HIV-1_35_AD', 22),
 ('HIV-1_ACD', 22),
 ('HIV-1_U', 21),
 ('HIV-1_01C', 19),
 ('HIV-1_02A', 19),
 ('HIV-1_22_01A1', 18),
 ('HIV-1_42_BF', 17),
 ('HIV-1_02G', 15),
 ('HIV-1_06_cpx', 14),
 ('HIV-1_12_BF', 14),
 ('HIV-1_BG', 14),
 ('HIV-1_14_BG', 13),
 ('HIV-1_71_BF1', 13),
 ('HIV-1_02B', 13),
 ('HIV-1_63_02A', 13),
 ('HIV-1_F2', 11),
 ('HIV-1_N', 11),
 ('HIV-1_85_BC', 11),
 ('HIV-1_H', 10),
 ('HIV-1_13_cpx', 10),
 ('HIV-1_0206', 9),
 ('HIV-1_59_01B', 9),
 ('HIV-1_64_BC', 9),
 ('HIV-1_

In [54]:
labels_combine = combine_labels(labels_sorted)

In [55]:
labels_combine

[('HIV-1_B', 5703),
 ('HIV-1_C', 2077),
 ('HIV-1_AE', 1425),
 ('HIV-1_-', 544),
 ('HIV-1_A1', 497),
 ('HIV-1_01B', 288),
 ('HIV-1_BC', 212),
 ('HIV-1_BF1', 179),
 ('HIV-1_AG', 160),
 ('HIV-1_cpx', 127),
 ('HIV-1_A6', 117),
 ('HIV-1_AD', 116),
 ('HIV-1_A1C', 111),
 ('HIV-1_A1D', 102),
 ('HIV-1_G', 95),
 ('HIV-1_D', 86),
 ('HIV-1_BF', 82),
 ('HIV-1_F1', 75),
 ('HIV-1_CD', 67),
 ('HIV-1_A1CD', 62),
 ('HIV-1_O', 57),
 ('HIV-1_0107', 56),
 ('HIV-1_01BC', 50),
 ('HIV-1_BG', 41),
 ('HIV-1_02A', 32),
 ('HIV-1_02A1', 26),
 ('HIV-1_ACD', 22),
 ('HIV-1_01C', 22),
 ('HIV-1_U', 21),
 ('HIV-1_02G', 20),
 ('HIV-1_01A1', 18),
 ('HIV-1_02B', 16),
 ('HIV-1_F2', 11),
 ('HIV-1_N', 11),
 ('HIV-1_H', 10),
 ('HIV-1_0206', 9),
 ('HIV-1_A', 8),
 ('HIV-1_02A3', 7),
 ('HIV-1_J', 7),
 ('HIV-1_AB', 7),
 ('HIV-1_A2D', 7),
 ('HIV-1_02D', 6),
 ('HIV-1_0209', 6),
 ('HIV-1_0222', 6),
 ('HIV-1_02A6', 6),
 ('HIV-1_CU', 5),
 ('HIV-1_A1U', 5),
 ('HIV-1_A1G', 5),
 ('HIV-1_P', 5),
 ('HIV-1_A1B', 5),
 ('HIV-1_02F2', 5),
 ('HI

In [56]:
len(labels_sorted)

242

In [59]:
len(labels_combine)

156

In [61]:
labels_lg10 = get_lgn(labels_combine,10)

In [63]:
len(labels_lg10)

35

In [67]:
labels_lg10_nocom = get_lgn(labels_sorted,18)

In [68]:
len(labels_lg10_nocom)

31

In [69]:
labels_lg10_nocom

[('HIV-1_B', 5703),
 ('HIV-1_C', 2077),
 ('HIV-1_01_AE', 1425),
 ('HIV-1_-', 544),
 ('HIV-1_A1', 497),
 ('HIV-1_01B', 205),
 ('HIV-1_02_AG', 160),
 ('HIV-1_BF1', 143),
 ('HIV-1_A6', 117),
 ('HIV-1_A1C', 111),
 ('HIV-1_G', 95),
 ('HIV-1_A1D', 94),
 ('HIV-1_AD', 94),
 ('HIV-1_BC', 91),
 ('HIV-1_D', 86),
 ('HIV-1_F1', 75),
 ('HIV-1_A1CD', 62),
 ('HIV-1_CD', 61),
 ('HIV-1_O', 57),
 ('HIV-1_01BC', 50),
 ('HIV-1_0107', 49),
 ('HIV-1_07_BC', 41),
 ('HIV-1_08_BC', 35),
 ('HIV-1_02A1', 26),
 ('HIV-1_11_cpx', 25),
 ('HIV-1_35_AD', 22),
 ('HIV-1_ACD', 22),
 ('HIV-1_U', 21),
 ('HIV-1_01C', 19),
 ('HIV-1_02A', 19),
 ('HIV-1_22_01A1', 18)]

In [62]:
labels_lg10

[('HIV-1_B', 5703),
 ('HIV-1_C', 2077),
 ('HIV-1_AE', 1425),
 ('HIV-1_-', 544),
 ('HIV-1_A1', 497),
 ('HIV-1_01B', 288),
 ('HIV-1_BC', 212),
 ('HIV-1_BF1', 179),
 ('HIV-1_AG', 160),
 ('HIV-1_cpx', 127),
 ('HIV-1_A6', 117),
 ('HIV-1_AD', 116),
 ('HIV-1_A1C', 111),
 ('HIV-1_A1D', 102),
 ('HIV-1_G', 95),
 ('HIV-1_D', 86),
 ('HIV-1_BF', 82),
 ('HIV-1_F1', 75),
 ('HIV-1_CD', 67),
 ('HIV-1_A1CD', 62),
 ('HIV-1_O', 57),
 ('HIV-1_0107', 56),
 ('HIV-1_01BC', 50),
 ('HIV-1_BG', 41),
 ('HIV-1_02A', 32),
 ('HIV-1_02A1', 26),
 ('HIV-1_ACD', 22),
 ('HIV-1_01C', 22),
 ('HIV-1_U', 21),
 ('HIV-1_02G', 20),
 ('HIV-1_01A1', 18),
 ('HIV-1_02B', 16),
 ('HIV-1_F2', 11),
 ('HIV-1_N', 11),
 ('HIV-1_H', 10)]

In [29]:
labels[9]

'HIV-1_B'

In [25]:
len(seq)

9680

In [26]:
label_id

'HIV-1_C'

In [9]:
ls = label_id.split('.')

In [10]:
ls

['C', 'IN', '1993', '93IN101', 'HIV-1']

In [18]:
'>' + '_'.join((ls[-1], ls[0]))

'>HIV-1_C'

In [ ]:
'_'.join((labe))

In [10]:
type(seq)

Bio.Seq.Seq

In [11]:
seq_str = str(seq)

In [12]:
type(seq_str)

str

In [41]:
a = np.array([1,2,3,4])

In [42]:
b = np.array([5,6,7,8])

In [43]:
np.hstack((a,b))

array([1, 2, 3, 4, 5, 6, 7, 8])

In [44]:
np.hstack?

In [8]:
a.upper()

'AA'

In [14]:
seq_hiv  = seq_str.replace('-','')

In [16]:
len(seq_hiv)

9680

In [2]:
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [3]:
import torch

In [30]:
from collections import defaultdict

In [35]:
from sklearn.metrics import classification_report, confusion_matrix

In [36]:
 classification_report?

In [9]:
import time
import datetime

def echo(msg):
        print("[%s] %s" % (datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), str(msg)))

def read_dic(dic_file):
    labels_dic = {}
    with(open(dic_file, 'r')) as fh:
        for line in fh:
            label, label_id = line.strip().split()
            labels_dic[label_id] = label
       
    return labels_dic

In [10]:
y_true = './y_true.npy'
y_pred = './y_pred.npy'
labels_dict = './labels_dict.txt'

In [11]:
y_true = np.load(y_true)
y_pred = np.load(y_pred)

labels_dict = read_dic(labels_dict)

In [12]:
y_true = [labels_dict[str(i)] for i in y_true]
y_pred = [labels_dict[str(j)] for j in y_pred]
c_names = np.unique(y_true)

In [13]:
print(c_names)

conf_matrix = confusion_matrix(y_true, y_pred,labels=c_names)
class_report = classification_report(y_true,y_pred)

['HIV1_01B' 'HIV1_BC' 'HIV1_cpx']


In [14]:
conf_matrix

array([[2, 0, 1],
       [1, 1, 0],
       [0, 1, 2]], dtype=int64)

In [16]:
print(class_report)

              precision    recall  f1-score   support

    HIV1_01B       0.67      0.67      0.67         3
     HIV1_BC       0.50      0.50      0.50         2
    HIV1_cpx       0.67      0.67      0.67         3

    accuracy                           0.62         8
   macro avg       0.61      0.61      0.61         8
weighted avg       0.62      0.62      0.62         8



In [ ]:
def main(parse):
    y_true = parse.t 
    y_pred = parse.p
    labels_dict = parse.d 

    y_true = np.load(y_true)
    y_pred = np.load(y_pred)

    labels_dict = read_dic(labels_dict)

    y_true = [labels_dict[str(i)] for i in y_true]
    y_pred = [labels_dict[str(j)] for j in y_pred]
    c_names = np.unique(y_true)

print(c_names)

conf_matrix = confusion_matrix(y_true, y_pred,labels=c_names)
class_report = classification_report(y_true,y_pred)

echo("The confusion matrix is: ")
print(conf_matrix)
echo("The classification report is: ")
print(class_report)
echo("Done")

In [28]:
def get_sample_weights(labelsFile):
    labels = np.load(labelsFile)

    labels_dict = defaultdict(list)

    for i, v in enumerate(labels):
        labels_dict[v].append(i)

    labels_num = [(k,len(v)) for k, v in labels_dict.items()]

    labels_num_sort = sorted(labels_num, key=lambda x:x[1], reverse=True)
    
    return labels_num_sort, len(labels)

In [33]:
labelsFile = './trainData/30mer_500_labels.npy'

In [36]:
labels,num = get_sample_weights(labelsFile)

In [35]:
labels

([('HIV1_cpx', 1538),
  ('HIV1_01B', 1063),
  ('HIV1_BC', 822),
  ('HIV1_BF', 577),
  ('HIV1_BF1', 511),
  ('HIV1_B', 386),
  ('HIV1_C', 371),
  ('HIV1_D', 274),
  ('HIV1_AG', 263),
  ('HIV1_AE', 261),
  ('HIV1_G', 250),
  ('HIV1_A1', 247),
  ('HIV1_BG', 242),
  ('HIV1_F1', 241),
  ('HIV1_O', 241),
  ('HIV1_H', 125),
  ('HIV1_02A1', 125),
  ('HIV1_F2', 102),
  ('HIV1_N', 88),
  ('HIV1_A2D', 86),
  ('HIV1_A1D', 85),
  ('HIV1_A6', 74),
  ('HIV1_CPZ', 74),
  ('HIV1_J', 71),
  ('HIV1_CD', 69),
  ('HIV1_AD', 68),
  ('HIV1_AU', 56),
  ('HIV1_C2U', 55),
  ('HIV1_02G', 54),
  ('HIV1_AB', 53),
  ('HIV1_0107', 53),
  ('HIV1_A2', 52),
  ('HIV1_DF', 52),
  ('HIV1_P', 37),
  ('HIV1_GOR', 36),
  ('HIV1_K', 34),
  ('HIV1_06A1', 34),
  ('HIV1_01A1', 33),
  ('HIV1_A4', 19),
  ('HIV1_A5U', 19),
  ('HIV1_06A6', 18),
  ('HIV1_02A', 18),
  ('HIV1_A3', 17)],
 8894)

In [38]:
len(labels)

43

In [11]:
labels2, num = get_sample_weights('./trainData/15mer_500_labels.npy')

In [13]:
num

8894

In [14]:
1538/8894

0.17292556779851587

In [12]:
labels2

[('HIV1_cpx', 1538),
 ('HIV1_01B', 1063),
 ('HIV1_BC', 822),
 ('HIV1_BF', 577),
 ('HIV1_BF1', 511),
 ('HIV1_B', 386),
 ('HIV1_C', 371),
 ('HIV1_D', 274),
 ('HIV1_AG', 263),
 ('HIV1_AE', 261),
 ('HIV1_G', 250),
 ('HIV1_A1', 247),
 ('HIV1_BG', 242),
 ('HIV1_F1', 241),
 ('HIV1_O', 241),
 ('HIV1_H', 125),
 ('HIV1_02A1', 125),
 ('HIV1_F2', 102),
 ('HIV1_N', 88),
 ('HIV1_A2D', 86),
 ('HIV1_A1D', 85),
 ('HIV1_A6', 74),
 ('HIV1_CPZ', 74),
 ('HIV1_J', 71),
 ('HIV1_CD', 69),
 ('HIV1_AD', 68),
 ('HIV1_AU', 56),
 ('HIV1_C2U', 55),
 ('HIV1_02G', 54),
 ('HIV1_AB', 53),
 ('HIV1_0107', 53),
 ('HIV1_A2', 52),
 ('HIV1_DF', 52),
 ('HIV1_P', 37),
 ('HIV1_GOR', 36),
 ('HIV1_K', 34),
 ('HIV1_06A1', 34),
 ('HIV1_01A1', 33),
 ('HIV1_A4', 19),
 ('HIV1_A5U', 19),
 ('HIV1_06A6', 18),
 ('HIV1_02A', 18),
 ('HIV1_A3', 17)]

In [37]:
a = np.load('./trainData/9mer_200_labels.npy')

In [41]:
labels, num = get_sample_weights('./trainData/9mer_200_labels.npy')

In [45]:
labels2 , num2 = get_sample_weights('./trainData/20mer_500_labels.npy')

In [46]:
num2

8894

In [42]:
num

22650

In [43]:
len(labels)

108

In [44]:
labels

[('HIV1_B', 985),
 ('HIV1_C', 935),
 ('HIV1_D', 702),
 ('HIV1_02_AG', 670),
 ('HIV1_01_AE', 663),
 ('HIV1_G', 636),
 ('HIV1_A1', 626),
 ('HIV1_F1', 614),
 ('HIV1_O', 612),
 ('HIV1_83_cpx', 451),
 ('HIV1_85_BC', 398),
 ('HIV1_90_BF1', 388),
 ('HIV1_71_BF1', 349),
 ('HIV1_59_01B', 348),
 ('HIV1_69_01B', 323),
 ('HIV1_H', 318),
 ('HIV1_63_02A1', 317),
 ('HIV1_55_01B', 305),
 ('HIV1_64_BC', 266),
 ('HIV1_09_cpx', 262),
 ('HIV1_58_01B', 262),
 ('HIV1_F2', 258),
 ('HIV1_82_cpx', 246),
 ('HIV1_06_cpx', 239),
 ('HIV1_12_BF', 229),
 ('HIV1_N', 223),
 ('HIV1_25_cpx', 221),
 ('HIV1_57_BC', 220),
 ('HIV1_50_A1D', 219),
 ('HIV1_72_BF1', 217),
 ('HIV1_18_cpx', 211),
 ('HIV1_CPZ', 188),
 ('HIV1_08_BC', 188),
 ('HIV1_A6', 187),
 ('HIV1_45_cpx', 183),
 ('HIV1_04_cpx', 181),
 ('HIV1_J', 180),
 ('HIV1_40_BF', 180),
 ('HIV1_61_BC', 180),
 ('HIV1_65_cpx', 179),
 ('HIV1_33_01B', 178),
 ('HIV1_14_BG', 177),
 ('HIV1_07_BC', 177),
 ('HIV1_11_cpx', 177),
 ('HIV1_15_01B', 177),
 ('HIV1_94_cpx', 177),
 ('HIV1_38_

In [20]:
好 = [0, 0,1]
善 = [0,0,0.8]
坏 = [0,0, -1]
恶 = [0,0,-0.6]

In [25]:
np.cos([np.array(好),np.array(善)])

array([[1.        , 1.        , 0.54030231],
       [1.        , 1.        , 0.69670671]])

In [19]:
b

[0, 0, 0.8]

In [4]:
30000/31

967.741935483871

In [9]:
41958*0.1/125

33.5664

In [40]:
s = '  abcd efg   '

In [42]:
s.strip().split()

['abcd', 'efg']

In [45]:
float(str(3.2)) / 4

0.8

In [72]:
def create_dict(filedir):
    desire_labels = {}
    with(open(filedir, 'r')) as fh:
        for line in fh:
            
            ls = line.strip().split()
           
            desire_labels[ls[0]] = float(ls[1])

    return desire_labels

In [73]:
desire_labels = './hiv_12825/dict_desire_labels.txt'

In [74]:
labels_dict = create_dict(desire_labels)

In [76]:
labels_dict.keys()

dict_keys(['HIV-1_B', 'HIV-1_C', 'HIV-1_01_AE', 'HIV-1_-', 'HIV-1_A1', 'HIV-1_01B', 'HIV-1_02_AG', 'HIV-1_BF1', 'HIV-1_A6', 'HIV-1_A1C', 'HIV-1_G', 'HIV-1_A1D', 'HIV-1_AD', 'HIV-1_BC', 'HIV-1_D', 'HIV-1_F1', 'HIV-1_A1CD', 'HIV-1_CD', 'HIV-1_O', 'HIV-1_01BC', 'HIV-1_0107', 'HIV-1_07_BC', 'HIV-1_08_BC', 'HIV-1_02A1', 'HIV-1_11_cpx', 'HIV-1_35_AD', 'HIV-1_ACD', 'HIV-1_U', 'HIV-1_01C', 'HIV-1_02A', 'HIV-1_22_01A1', 'HIV-1_42_BF', 'HIV-1_02G', 'HIV-1_06_cpx', 'HIV-1_12_BF', 'HIV-1_BG', 'HIV-1_14_BG', 'HIV-1_71_BF1', 'HIV-1_02B', 'HIV-1_63_02A', 'HIV-1_F2', 'HIV-1_N', 'HIV-1_85_BC', 'HIV-1_H', 'HIV-1_13_cpx'])

In [79]:
'HIV-1_B' not in set(labels_dict.keys())

False

In [ ]:
def split(data, step, labels_dict):

    desire_labels = set(labels_dict.keys())

    line = 0
    for(line=0;line < len(data); line +=2):
        label = data[line]
        if label.replace('>','') not in desire_labels:
            continue
        seq = data[line+1]
        num = int(len(seq)/step)

        for i in range(num):			
            fh.write(label)
            fh.write('\n')
            fh.write(seq[i*step:(i+1)*step])
            fh.write('\n')

    fh.close()

In [84]:
l = 10

In [88]:
list(range(0,l,step=2))

TypeError: range() does not take keyword arguments

In [93]:
list(range(0, l ,2))

[0, 2, 4, 6, 8]

In [94]:
for line in range(0,10, 2):
    print(line)

0
2
4
6
8


In [32]:
conf_matrix = np.loadtxt('./confusion_matrix.txt')

In [36]:
conf_matrix[0].astype(int)

array([1931,    0,    0,    0,    4,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    1,    0,    2,   39,   48,
          0,    0,    0,    0,    0,    0,    0,    0,   74,    2,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0])

In [40]:
np.loadtxt?

In [41]:
c_names  = np.loadtxt('./c_names.txt',dtype=str)

In [42]:
c_names

array(['HIV-1_C', 'HIV-1_B', 'HIV-1_01_AE', 'HIV-1_O', 'HIV-1_-',
       'HIV-1_06_cpx', 'HIV-1_63_02A', 'HIV-1_02A1', 'HIV-1_12_BF',
       'HIV-1_G', 'HIV-1_U', 'HIV-1_11_cpx', 'HIV-1_F1', 'HIV-1_A6',
       'HIV-1_14_BG', 'HIV-1_BF1', 'HIV-1_08_BC', 'HIV-1_07_BC',
       'HIV-1_D', 'HIV-1_01BC', 'HIV-1_BC', 'HIV-1_CD', 'HIV-1_02_AG',
       'HIV-1_01B', 'HIV-1_A1D', 'HIV-1_71_BF1', 'HIV-1_BG',
       'HIV-1_42_BF', 'HIV-1_0107', 'HIV-1_02G', 'HIV-1_A1C',
       'HIV-1_A1CD', 'HIV-1_02B', 'HIV-1_01C', 'HIV-1_AD', 'HIV-1_02A',
       'HIV-1_H', 'HIV-1_22_01A1', 'HIV-1_A1', 'HIV-1_ACD', 'HIV-1_35_AD',
       'HIV-1_85_BC', 'HIV-1_13_cpx', 'HIV-1_N', 'HIV-1_F2'], dtype='<U13')

In [43]:
 np.random.randn(10, 5)
    

array([[ 3.90698555e-01, -1.24224081e+00,  7.38564342e-01,
         1.16396019e+00, -5.12312182e-01],
       [ 8.63797320e-01, -1.73056352e+00,  2.09940823e-01,
        -2.18971498e+00, -1.37735477e-01],
       [ 1.81571451e-03, -6.05731713e-01, -2.42076676e+00,
        -4.06545690e-01,  1.33727850e+00],
       [ 9.13144770e-01, -4.51080205e-01, -2.43272612e+00,
        -7.06747964e-01,  7.27693202e-01],
       [-5.24144217e-01, -1.15287787e-01, -6.41402845e-01,
        -1.43864802e+00, -1.25110270e+00],
       [-2.39919775e-01,  6.82599871e-01,  8.29803113e-01,
        -2.43508954e+00,  1.45885602e-01],
       [ 1.04120672e-03,  5.83842774e-01, -3.46985370e-01,
         8.87572273e-01, -8.48938773e-01],
       [-6.80295321e-02, -3.68493404e-01, -2.13568905e-01,
        -3.11030749e+00, -5.11962724e-01],
       [-8.71985870e-01, -1.12664674e+00,  4.99485018e-01,
        -1.61755008e+00,  7.00321139e-01],
       [ 1.87416722e+00, -1.11527178e+00,  5.93845114e-01,
         6.35501701e-01

In [46]:
pd.DataFrame?

In [50]:
ls = list(range(20))

In [51]:
['index %d'%i for i in ls]

['index 0',
 'index 1',
 'index 2',
 'index 3',
 'index 4',
 'index 5',
 'index 6',
 'index 7',
 'index 8',
 'index 9',
 'index 10',
 'index 11',
 'index 12',
 'index 13',
 'index 14',
 'index 15',
 'index 16',
 'index 17',
 'index 18',
 'index 19']

In [15]:
import torch.nn.functional as F

In [13]:
a = torch.randn(2,10)

In [14]:
a

tensor([[ 0.3711,  0.2606,  0.0462, -1.8225, -2.0624, -0.2429,  1.3283,  0.7921,
         -0.5128,  1.1057],
        [-1.6105,  0.6885, -0.2127,  1.7746, -1.5326,  0.4018,  0.5021, -0.8331,
          0.1575,  0.8281]])

In [16]:
preds = F.softmax(a,dim=1)

In [17]:
preds

tensor([[0.1002, 0.0897, 0.0724, 0.0112, 0.0088, 0.0542, 0.2608, 0.1526, 0.0414,
         0.2088],
        [0.0124, 0.1232, 0.0500, 0.3651, 0.0134, 0.0925, 0.1023, 0.0269, 0.0725,
         0.1417]])

In [28]:
most = torch.argsort(preds,dim=1, descending=True)

In [27]:
most

tensor([[4, 3, 8, 5, 2, 1, 0, 7, 9, 6],
        [0, 4, 7, 2, 8, 5, 6, 1, 9, 3]])

In [29]:
most

tensor([[6, 9, 7, 0, 1, 2, 5, 8, 3, 4],
        [3, 9, 1, 6, 5, 8, 2, 7, 4, 0]])

In [44]:
most[0:1]

tensor([[6, 9, 7, 0, 1, 2, 5, 8, 3, 4]])

In [73]:
most_np = most.numpy()

In [76]:
most_np[0]

array([6, 9, 7, 0, 1, 2, 5, 8, 3, 4], dtype=int64)

In [86]:
most_np[: ,:3]

array([[6, 9, 7],
       [3, 9, 1]], dtype=int64)

In [87]:
most[ : , :3]

tensor([[6, 9, 7],
        [3, 9, 1]])

In [49]:
valid_acc = 0.1222233434444545

In [65]:
print('Validation accuracy :  %.3f'  %        (valid_acc*100),"%")

Validation accuracy :  12.222 %


In [68]:
timeStamp = time.strftime("%Y-%m-%d-%H_%M_%S_evaluate",time.localtime(time.time())) 

In [69]:
timeStamp

'2020-07-17-13_25_13_evaluate'

In [70]:
a = None

In [71]:
a == None

True

In [88]:
np.hstack?

In [90]:
np.concatenate?

In [91]:
a = np.arange(10)

In [92]:
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [93]:
b = np.arange(20,23)

In [94]:
b

array([20, 21, 22])

In [95]:
np.concatenate((a,b))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 20, 21, 22])

In [101]:
ax = torch.randn(3,5)

In [102]:
ax

tensor([[ 0.2806,  0.1663, -0.0304,  1.2933, -0.7180],
        [-1.6365,  1.9176, -0.9852,  0.6688,  1.4465],
        [-0.3283,  0.4593,  0.6410, -1.3690,  1.2950]])

In [109]:
ab = torch.randn(2,5)

In [110]:
ab

tensor([[-0.2925, -0.1121,  1.2324,  0.2043,  0.6998],
        [-0.8558, -0.3455, -1.6128,  1.2651,  0.4137]])

In [124]:
ax = np.empty(shape=[0, 5],dtype=int)

In [125]:
ax

array([], shape=(0, 5), dtype=int32)

In [126]:
np.concatenate((ax,ab.numpy()))

array([[-0.29251522, -0.11210923,  1.23240864,  0.20431504,  0.6997664 ],
       [-0.8558498 , -0.34546679, -1.61279571,  1.26505792,  0.41371825]])

In [13]:
prob = np.load('./result/y_prob_2020-07-17-15_58_05_evaluate.npy')

In [14]:
true_labels = np.load('./result/y_true_2020-07-17-15_58_05_evaluate.npy')

In [28]:
names = []
with open('./c_names.txt', 'r') as fh:
    for line in fh:
        names.append(line.strip())
        
names = np.array(names)

In [15]:
true_labels[32]

4

In [16]:
len(prob[0])

45

In [136]:
prob[32][9]

8.168635818488808e-15

In [137]:
import pandas as pd

In [17]:
df1 = pd.DataFrame(np.random.randint(low=0, high=10, size=(5,5)))

In [18]:
df1

,0,1,2,3,4
0,9,0,0,4,2
1,9,2,9,0,7
2,9,9,4,0,0
3,7,7,9,3,6
4,8,4,4,4,6


In [141]:
a = list(range(10))

In [142]:
b = [(i, i/10) for i in a]

In [149]:
df2 = pd.DataFrame(np.concatenate((b,b)))

In [20]:
prob_t = torch.Tensor(prob)

In [21]:
prob_sort = torch.argsort(prob_t[32:34],dim=1,descending=True)

In [23]:
top3 = prob_sort[:, :3]

In [36]:
top3

tensor([[ 4, 29,  6],
        [ 3,  5, 12]])

In [25]:
prob_t[32][top3[0]]

tensor([0.7270, 0.1890, 0.0504])

In [30]:
names_sort = names[top3[0]]

In [31]:
names_sort

array(['HIV-1_-', 'HIV-1_02G', 'HIV-1_63_02A'], dtype='<U13')

In [68]:
prob_t[32:34][:, :3]

tensor([[3.9796e-19, 7.6382e-10, 1.1568e-08, 4.3577e-08, 7.2698e-01, 1.2823e-05,
         5.0424e-02, 5.5206e-07, 1.1645e-09, 8.1686e-15, 1.5109e-05, 5.0939e-25,
         1.4037e-07, 4.4529e-18, 1.8481e-17, 8.5444e-10, 2.0617e-15, 3.3958e-15,
         7.2402e-15, 8.5379e-11, 1.5187e-08, 2.2341e-13, 2.1166e-08, 3.1434e-05,
         1.3359e-14, 2.1661e-06, 8.4783e-18, 3.2653e-14, 3.3038e-02, 1.8899e-01,
         4.9670e-04, 2.9056e-10, 3.2516e-12, 1.0026e-18, 4.6945e-06, 1.2446e-06,
         2.9442e-06, 7.9691e-20, 2.8921e-12, 5.4052e-25, 1.0132e-19, 8.0509e-15,
         3.1458e-23, 2.9156e-20, 1.0055e-13],
        [3.6155e-23, 2.9685e-13, 5.0864e-14, 9.9995e-01, 4.8674e-22, 3.2380e-05,
         8.8903e-14, 1.3651e-17, 1.9825e-13, 1.2832e-20, 1.1123e-24, 6.2164e-41,
         8.5808e-06, 1.1648e-18, 1.4440e-13, 2.6054e-14, 2.6752e-32, 4.7480e-34,
         2.0962e-19, 9.3990e-25, 1.1936e-25, 7.3386e-33, 3.6431e-13, 1.2199e-08,
         2.9618e-06, 1.2139e-24, 0.0000e+00, 9.5294e-09, 7.1827

In [69]:
prob_t[32:34][:, :3]

tensor([[3.9796e-19, 7.6382e-10, 1.1568e-08],
        [3.6155e-23, 2.9685e-13, 5.0864e-14]])

In [ ]:
for i in range(len(top3)):
    line = []
    

In [56]:
pd.DataFrame?

In [97]:
pd3 = pd.DataFrame(data=data,columns=['TOP1','p','TOP2','p','TOP3','p'])

In [98]:
pd3

,TOP1,p,TOP2,p,TOP3,p
0,HIV-1_-,0.727,HIV-1_02G,0.189,HIV-1_63_02A,0.050
1,HIV-1_O,1.000,HIV-1_06_cpx,0.000,HIV-1_F1,0.000


In [47]:
a = np.array( [1,2,3])
b = np.array(['a', 'b', 'c'])

In [41]:
np.concatenate((a,b))

array(['1', '2', '3', 'a', 'b', 'c'], dtype='<U11')

In [70]:
v = prob_t[32:34]

In [75]:
v[0][top3[0]]

tensor([0.7270, 0.1890, 0.0504])

In [102]:
ls = torch.random.randlen(prob)

212849

In [105]:
ls = np.random.randint?

In [111]:
ls = np.random.randint(0,212849,100)

In [112]:
ls

array([ 63926, 133566,  46628,  50837,  32720, 181051, 138029, 165405,
        69947, 148810, 192026,  44385, 163340, 155334, 194632, 142473,
       190366, 104841,  59184, 110540, 156293,   9159, 177232,  44667,
        93481,  21732,  27429, 183099, 139598,  48636, 154628, 162849,
        99504,  69520, 128032, 128222, 186122, 180744,  80464, 159962,
       135231, 165578, 110812,  93810, 165983, 156430,  98754, 169291,
        74238, 168025,   3208,  71608, 109502,  78352,  34785,  16164,
        23654, 189664,  98132,  61945,  25638, 162323,  74828,  12932,
       148373,  53377, 171260, 130707,  25575, 186900, 173271, 118386,
        97513, 141467,  11948,  38003, 147424,  95361, 162139,  33000,
        71530, 159529,  47864, 187170, 125463,  13481, 102320, 121801,
       113400,  87166, 119178, 127640,  36670,  60738,  71464, 118253,
        17702, 120652, 110403, 141958])

In [125]:
def creat_frame(prob, true_labels,names):
    prob_tensor = torch.Tensor(prob)[ls]
    v = prob[ls]
    prob_sort = torch.argsort(prob_tensor,dim=1,descending=True)
    top3 = prob_sort[:, :3]
    data = []
    
    for j in range(len(top3)):
        a = names[top3[j]]
        b = v[j][top3[j]]
        line = []
        for i in range(3):
            line.append(a[i])
            line.append('%.3f'%b[i])
        data.append(line)
        
    return data
    
    

In [126]:
data = creat_frame(prob, true_labels,names)

In [134]:
index = names[true_labels[ls]]

In [130]:
pd.DataFrame?

In [135]:
pd3 = pd.DataFrame(data=data,columns=['TOP1','p','TOP2','p','TOP3','p'], index=index)

In [137]:
pd3.to_csv('./result/data_frame.csv')

In [3]:
columns = []
for i in range(3):
    columns.append('TOP%d'%(i+1))
    columns.append('p')

In [4]:
columns

['TOP1', 'p', 'TOP2', 'p', 'TOP3', 'p']

In [95]:
for j in range(len(top3)):
    a = names[top3[j]]
    b = v[j][top3[j]]
    
    line = []
    for i in range(3):
        line.append(a[i])
        line.append('%.3f'%b[i].numpy())
    print(line)
    data.append(line)

['HIV-1_-', '0.727', 'HIV-1_02G', '0.189', 'HIV-1_63_02A', '0.050']
['HIV-1_O', '1.000', 'HIV-1_06_cpx', '0.000', 'HIV-1_F1', '0.000']


In [96]:
data = np.array(data)

In [57]:
line = np.array([line])

In [58]:
line.shape

(1, 6)

In [59]:
line

array([['a', '1', 'b', '2', 'c', '3']], dtype='<U1')

In [7]:
def build_names(c_names):
	names = []
	with open(c_names, 'r') as fh:
	    for line in fh:
	        names.append(line.strip())
	        
	return np.array(names)

In [8]:
build_names('./c_names.txt')

NameError: name 'np' is not defined

In [ ]:
tor